In [3]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-13.6")
user

UserTO(id=909e545b-7a4a-40e2-8ed1-de17e5e37145, username=Johann-13.6)

In [4]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


UserTO(id=e9f56c4b-da9e-4a84-9d43-b3045c5d377f, username=Johann-13.5)

In [5]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=2139c063-07a1-4720-ae3a-aa6288ff5449, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=5a7b80b6-472d-4476-ada5-ece0bcd11732, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=08bb0915-4a52-4abb-90be-97433697fb18, name=ING RON, unit=Currency(value=RON))
AccountTO(id=f6905a3d-77d8-4314-94ce-fcb0d0d1e89e, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=31fa53c2-af28-440e-aec0-3aa36f00a8c8, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=158b1236-b0e0-4327-83ac-c19516766632, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=1bad7d2b-6c34-4fb9-9704-11b16234af4a, name=Other, unit=Currency(value=RON))
AccountTO(id=49cc2828-ecfd-4b69-9934-6fe272a034b7, name=BT RON, unit=Currency(value=RON))
AccountTO(id=4d4c5300-3317-43bb-b7e3-b3b136c235f3, name=BT Economy RON, unit=Currency(value=RON))

In [6]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=aaec4d88-b8c0-4ad6-879c-114a8d47b9f1, name=Expenses)
FundTO(id=e391ec09-4296-4f3e-aca6-d259fa7858f8, name=Work Income)
FundTO(id=334f9ecb-3c69-422d-aded-d9f60d65c0ac, name=Savings)
FundTO(id=b3e1ffca-d02a-4c24-b502-34ebd4fb958f, name=Gifts)

In [7]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [8]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=96ae75f8-24d6-4294-a8ed-41dec9d71346, status=COMPLETED, reason=null)